In [1]:
import shutil
import math
import csv
import cv2
from PIL import Image, ImageFilter
import numpy as np
from pathlib import Path

In [2]:
base_dir = Path('/media/external/tcc_work')
base_dir.cwd()

PosixPath('/media/external/tcc_work/code')

In [3]:
def average_distance(img1, img2):
    return np.average(img1 - img2)

def l2_distance(img1, img2):
    width, height = img1.shape[:2]
    return np.linalg.norm(img1 - img2) / (width * height)

def structure_content(img1, img2):
    return np.sum(np.square(img1)) / np.sum(np.square(img2))

def image_fidelity(img1, img2):
    return 1 - np.sum(np.square(img1 - img2)) / np.sum(np.square(img2))

def n_cross_correlation(img1, img2):
    return np.sum(img1 * img2) / np.sum(np.square(img1))

def nmse(img1, img2):
    return np.sum(np.square(img1 - img2)) / np.sum(np.square(img1))

def psnr(img1, img2):
    return cv2.PSNR(img1, img2)

In [4]:
def filt(sigma=0.5,mask=3):
  """
    Create the kernel (numpy array) for the Gaussian smoothing 
    operation used by Avcibas et al.
  """
  K = 1/(2*math.pi*sigma**2)
  assert mask % 2 == 1
  mx = int(mask/2)
  return np.array( [ [ K*math.exp( -(m**2+n**2)/(2*sigma**2) )
     for m in range(-mx,mx+1) ]
     for n in range(-mx,mx+1) ] )

In [5]:
temp_dir = base_dir / 'temp_jpg_filter'

def create_filter_copy(jpg_file):
  filename = jpg_file.stem
  img = Image.open(jpg_file)
  
  filter = ImageFilter.Kernel((3, 3), filt(sigma=0.5,mask=3).flatten())
  filtered_img = img.filter(filter)

  output_path = temp_dir / f'{filename}.jpg'
  filtered_img.save(output_path)
  return output_path

In [9]:
def write_features_and_labels(folder):
  if temp_dir.exists():
    shutil.rmtree(temp_dir)
  temp_dir.mkdir()

  csv_path = base_dir / f'{folder.name}_features_label.csv'
  if csv_path.exists():
    csv_path.unlink()

  with csv_path.open('w', newline = '') as csv_file:
    writer = csv.writer(csv_file)
    for child in folder.iterdir():
      if child.suffix != '.jpg':
        continue
      if child.stat().st_size == 0:
        continue
      print(f'Processing file {child.as_posix()}')
      jpg_copy_path = create_filter_copy(child)
      img1 = cv2.imread(child.as_posix())
      img2 = cv2.imread(jpg_copy_path.as_posix())

      ad = average_distance(img1, img2)
      l2dist = l2_distance(img1, img2)
      struct_content = structure_content(img1, img2)
      fidelity = image_fidelity(img1, img2)
      n_cross_rel = n_cross_correlation(img1, img2)
      nmse_val = nmse(img1, img2)
      psnr_val = psnr(img1, img2)

      filename = child.stem

      if filename.startswith('encoded_'):
        label = 1
      else:
        label = 0
      
      writer.writerow([filename, ad, l2dist, struct_content, fidelity, n_cross_rel, nmse_val, psnr_val, label])

In [10]:
dataset_folder = base_dir / 'jsteg'
print(f'writing labels and features for {dataset_folder.name}')
write_features_and_labels(dataset_folder)

writing labels and features for jsteg
Processing file /media/external/tcc_work/jsteg/encoded_12614.jpg
